## **NiN网络 - 网络中的网络**

LeNet、AlexNet和VGG在设计上的共同之处是：先以由卷积层构成的模块充分抽取空间特征，再以由全连接层构成的模块来输出分类结果。其中，AlexNet和VGG对LeNet的改进主要在于如何对这两个模块加宽（增加通道数）和加深。

NiN提取了另外一个思路，即串联多个由卷积层和“全连接”层构成的小网络来构建一个深层网络。

### **NiN块**

卷积层的输入和输出通常是4维数组（样本，通道，高、宽），而全连接层的输入和输出则通常是二维数组（样本，特征）。如果想在全连接层后再接上卷积层，则需要将全连接层的输出变换为4维。$1\times 1$卷积层，它可以看成全连接层，其中空间维度（高和宽）上的每个元素相当于样本，通道相当于特征。因此，NiN使用 $1\times 1$卷积层来替代全连接层，从而使空间信息能够自然传递到后面的层中去。

<img src="NiN.png"></img>

NiN块是NiN中的基础块。它由一个卷积层加两个充当全连接层的$1\times 1$卷积层串联而成。其中，第一个卷积层的超参数可以自行设置，而第二个和第三个卷积层的超参数一般是固定的。


In [1]:
import d2lzh as d2l
from mxnet import gluon,init,nd
from mxnet.gluon import nn 

def nin_block(num_channels,kernel_size,strides,padding):
    blk = nn.Sequential()
    blk.add(nn.Conv2D(num_channels, kernel_size, strides, padding, activation = "relu"),
            nn.Conv2D(num_channels, kernel_size = 1, activation = "relu"),
            nn.Conv2D(num_channels, kernel_size = 1, activation = 'relu'))
    return blk

### **NiN模型**

NiN与AlexNet的卷积层设定有类似之处。NiN使用卷积窗口形状分别为 $11\times 11$、$5\times 5$ 和 $3\times 3$ 的卷积层，相应的输出通道数也与AlexNet中的一致。每个NiN后接一个步幅为2、窗口形状为$3\times 3$的最大池化。

NiN与AlexNet的不同：NiN去掉了AlexNet最后的3个全连接层，取而代之的，NiN使用了输出通道数等于标签类别数的NiN块，然后使用全局平均池化对每个通道中的所有元素求平均并直接用于分类。这里的全局平均池化层即窗口形状等于输入空间维形状的平均池化层。NiN的这个设计的好处是可以显著减小模型参数尺寸，从而缓解过拟合。然而，该设计有时会造成获得有效模型的训练时间的增加。



In [2]:
net = nn.Sequential()
net.add(nin_block(96, kernel_size = 11, strides = 4, padding = 0 ),
        nn.MaxPool2D(pool_size = 3, strides = 2), 
        nin_block(256, kernel_size = 5, strides = 1, padding = 2 ),
        nn.MaxPool2D(pool_size = 3, strides = 2),
        nin_block(384, kernel_size = 3, strides = 1, padding = 1),
        nn.MaxPool2D(pool_size = 3, strides = 2),
        nn.Dropout(0.5),
        # 标签类别数是10
        nin_block(10, kernel_size = 3, strides = 1, padding = 1),
        # 全局平均池化层将窗口形状自动设置成输入的高和宽
        nn.GlobalAvgPool2D(),
        # 将4维的输出转成2维的输出，其形状为(批大小，10)
        nn.Flatten())



In [3]:
# 构建一个数据样本来查看每一层的输出形状
X = nd.random.uniform(shape = (1,1,224,224))
net.initialize()
for layer in net:
    X = layer(X)
    print(layer.name,"output shape: \t",X.shape)

sequential1 output shape: 	 (1, 96, 54, 54)
pool0 output shape: 	 (1, 96, 26, 26)
sequential2 output shape: 	 (1, 256, 26, 26)
pool1 output shape: 	 (1, 256, 12, 12)
sequential3 output shape: 	 (1, 384, 12, 12)
pool2 output shape: 	 (1, 384, 5, 5)
dropout0 output shape: 	 (1, 384, 5, 5)
sequential4 output shape: 	 (1, 10, 5, 5)
pool3 output shape: 	 (1, 10, 1, 1)
flatten0 output shape: 	 (1, 10)


In [ ]:
# 训练模型
lr, num_epochs, batch_size, ctx = 0.1, 5, 10, d2l.try_gpu()

net.initialize(force_reinit=True, ctx = ctx, init = init.Xavier())
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})

train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize = 224)
d2l.train_ch5(net, train_iter , test_iter, batch_size, trainer, ctx, num_epochs)


NiN重复使用由卷积层和代替全连接层的 $1\times 1$卷积层构成的NiN块来构建深层网络。

NiN去除了容易造成过拟合的全连接输出层，而是将其替换成输出通道数等于标签类别数的NiN块和全局平均池化层。

NiN的以上设计思想影响了后面一系列卷积神经网络的设计。
